In [1]:
from dotenv import load_dotenv
load_dotenv()

True

1. 데이터 문서 전처리

`from langchain_teddynote.community.pinecone import preprocess_documents`

2. vectorstore index 생성

`from langchain_teddynote.community.pinecone import create_index`

3. sparse encoder생성

`from langchain_teddynote.community.pinecone import create_sparse_encoder`

4. db 인덱스에 update + insert (upsert) - contents와 metadatas를 추가

`from langchain_teddynote.community.pinecone import upsert_documents`

5. hybrid search

`from langchain_teddynote.community.pinecone import PineconeKiwiHybridRetrieverm`


# 데이터 전처리

In [2]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

split_docs = []

# 텍스트 파일을 load -> List[Document] 형태로 변환
files = sorted(glob.glob("data/*.pdf"))
for file in files:
    loader = PDFPlumberLoader(file)
    split_docs.extend(loader.load_and_split(text_splitter))

len(split_docs)

619

# 문서의 전처리

In [3]:
# 저장하고 싶은 메타데이터만 뽑는다.
from langchain_teddynote.community.pinecone import preprocess_documents

contents, metadatas = preprocess_documents(
    split_docs=split_docs,
    #metadata_keys=["source", "page", "author"],
    metadata_keys=["source", "page"],
    min_length=5, #문서의 길이를 가지고 filtering. 5글자 이상의 문서만을 사용. (예를들어 숫자만 있거나 제목만 있는경우) 줄어든것을 확인할 수 있다.
    use_basename=True # False인 경우 폴더 위치까지 모두 적혀있다.
)

  0%|          | 0/619 [00:00<?, ?it/s]

# 새로운 VectorStore 인덱스 생성
**주의사항**

- metric 은 유사도 측정 방법을 지정합니다. 만약 HybridSearch 를 고려하고 있다면 metric 은 dotproduct 로 지정합니다. 파인콘 홈페이지에서도 생성 가능 합니다.

In [4]:
import os
from langchain_teddynote.community.pinecone import create_index

pc_index = create_index(
    api_key=os.environ["PINECONE_API_KEY"],
    index_name="jaeho-note-index", # 인덱스 이름을 지정합니다.
    dimension=1536, # Embedding 차원과 맞춥니다. (OpenAIEmbeddings: 1536, UpstageEmbeddings: 4096)
    metric="dotproduct" # 유사도 측정 방법을 지정합니다. (dotproduct, euclidean, cosine) Hybrid search를 위해선 dotproduct를 사용
)

[create_index]
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'jaehonote-namespace': {'vector_count': 1458}},
 'total_vector_count': 1458}


In [5]:
# 아래는 유료 Pod 를 사용하는 예시입니다. 유료 Pod 는 무료 Serverless Pod 대비 더 확장된 기능을 제공합니다.

# 참고: https://docs.pinecone.io/guides/indexes/choose-a-pod-type-and-size
# import os
# from langchain_teddynote.community.pinecone import create_index
# from pinecone import PodSpec

# # Pinecone 인덱스 생성
# pc_index = create_index(
#     api_key=os.environ["PINECONE_API_KEY"],
#     index_name="teddynote-db-index2",  # 인덱스 이름을 지정합니다.
#     dimension=4096,  # Embedding 차원과 맞춥니다. (OpenAIEmbeddings: 1536, UpstageEmbeddings: 4096)
#     metric="dotproduct",  # 유사도 측정 방법을 지정합니다. (dotproduct, euclidean, cosine)
#     pod_spec=PodSpec(
#         environment="us-west1-gcp", pod_type="p1.x1", pods=1
#     ),  # 유료 Pod 사용
# )

# Sparse Encoder 생성

In [7]:
from langchain_teddynote.korean import stopwords
from langchain_teddynote.community.pinecone import create_sparse_encoder, fit_sparse_encoder

stopword = stopwords()
stopword

# 한글 불용어 사전 stopwords() + Kiwi 형태소 분석기
sparse_encoder = create_sparse_encoder(stopwords(), mode='kiwi') # 한글처리를 하기위한 kiwi

In [8]:
# Sparse Encoder를 사용하여 contents를 학습
save_path = fit_sparse_encoder(
    sparse_encoder=sparse_encoder, contents=contents, save_path="./sparse_encoder.pkl"
)

  0%|          | 0/619 [00:00<?, ?it/s]

[fit_sparse_encoder]
Saved Sparse Encoder to: ./sparse_encoder.pkl


[선택사항] 나중에 학습을 저장한 Sparse Encoder를 다시 불러올떄 사용하는 코드

In [9]:
from langchain_teddynote.community.pinecone import load_sparse_encoder

sparse_encoder = load_sparse_encoder("./sparse_encoder.pkl")

[load_sparse_encoder]
Loaded Sparse Encoder from: ./sparse_encoder.pkl


# Pinecone: DB Index에 추가 (Upsert)

In [10]:
#%%time
from langchain_teddynote.community.pinecone import upsert_documents
from langchain_openai import OpenAIEmbeddings
openai_embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
upsert_documents(
    index=pc_index, #picone 인덱스
    namespace="jaehonote-namespace-0", #파인콘 namespace. 비슷한 문서에는 같은 namespace에 저장
    contents=contents, # 이전에 처리한 문서내용
    metadatas=metadatas, # 이전에 처리한 문서 데이터 
    sparse_encoder=sparse_encoder, # Sparse Encoder
    embedder=openai_embeddings,
    batch_size=32,
    
)

  0%|          | 0/20 [00:00<?, ?it/s]

[upsert_documents]
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'jaehonote-namespace': {'vector_count': 1458},
                'jaehonote-namespace-0': {'vector_count': 608}},
 'total_vector_count': 2066}


분산처리

## 검색기(Retriever) 생성

### PineconeKiwiHybridRetriever 초기화 파라미터 설정

In [11]:
from langchain_teddynote.community.pinecone import init_pinecone_index

pinecone_params = init_pinecone_index(
    index_name="jaeho-note-index",  # Pinecone 인덱스 이름
    namespace="jaehonote-namespace",  # Pinecone Namespace
    api_key=os.environ["PINECONE_API_KEY"],  # Pinecone API Key
    sparse_encoder_path="./sparse_encoder.pkl",  # Sparse Encoder 저장경로(save_path)
    stopwords=stopwords(),  # 불용어 사전
    tokenizer="kiwi",
    embeddings=openai_embeddings,  # Dense Embedder
    top_k=5,  # Top-K 문서 반환 개수
    alpha=0.5,  # alpha=0.75로 설정한 경우, (0.75: Dense Embedding, 0.25: Sparse Embedding)
)

[init_pinecone_index]
{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'jaehonote-namespace': {'vector_count': 1458},
                'jaehonote-namespace-0': {'vector_count': 619}},
 'total_vector_count': 2077}


### PineconeKiwiHybridRetriever

`PineconeKiwiHybridRetriever` 클래스는 Pinecone과 Kiwi를 결합한 하이브리드 검색기를 구현합니다.

In [12]:
from langchain_teddynote.community.pinecone import PineconeKiwiHybridRetriever

# 검색기 생성
pinecone_retriever = PineconeKiwiHybridRetriever(**pinecone_params)

In [63]:
# 실행 결과
search_results = pinecone_retriever.invoke("gpt-4o 미니 출시 관련 정보에 대해서 알려줘")
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
갈릴레오의 LLM 환각 지수 평가에서 GPT-4가 가장 우수
KEY Contents
n 주요 LLM의 환각 현상을 평가한 ‘LLM 환각 지수’에 따르면 GPT-4는 작업 유형과 관계없이
가장 우수한 성능을 보였으며 GPT-3.5도 거의 동등한 성능을 발휘
n 오픈소스 모델 중에서는 메타의 라마2가 RAG 없는 질문과 답변 및 긴 형식의 텍스트
생성에서 가장 우수한 성능을 발휘
£주요 LLM 중 GPT-4가 가장 환각 현상 적고 GPT-3.5 터보도 비슷한 성능 기록
n 머신러닝 데이터 관리 기업 갈릴레오(Galileo)가 2023년 11월 15일 주요 LLM의 환각 현상을 평가한
‘LLM 환각 지수(LLM Hallucination Index)’를 발표
∙ 생성 AI의 환각 현상은 AI 시스템이 잘못된 정보를 생성하거나, 현실과 다른 부정확한 결과를 내놓는
현상으로, 기업의 AI 도입을 가로막는 주요 장애물이며, 환각 지수는 신뢰할 수 있는 생성 AI 구축을 위해
환각을 평가하고 측정하는 구조화된 접근방식을 제공
∙ 환각 지수는 △검색 증강 생성(Retrieval-Augmented Generation, RAG)*을 포함한 질문과 답변 △RAG
없는 질문과 답변 △긴 형식의 텍스트(보고서나 기사, 에세이) 생성의 3개 작업 유형에 대하여 환각을
기준으로 LLM의 순위를 평가
* 기존에 학습된 데이터가 아닌 외부 소스(데이터셋, 데이터베이스, 문서 등)에서 가져온 정보를 검색해 활용하는 기술
n 3개의 작업 유형 평가 전체에서 오픈AI의 GPT-4가 최고의 성능을 기록했으며, GPT-3.5 터보도
GPT-4와 거의 동등한 성능을 발휘
∙ 메타의 라마2(Llama-2-70b)는 RAG 없는 질문과 답변 유형에서 오픈소스 모델 가운데 가장 우수했고 긴
형식의 텍스트 생성에서도 GPT-4에 준하는 성능을 기록했으나, RAG 포함 질문과 답변에서는 허깅
페이스의 제퍼(Zephyr-7b)가 라마2를 능가
{'p

In [64]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽", search_kwargs={"alpha": 1, "k": 1}
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화
KEY Contents
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인
앤스로픽 및 오픈AI와 협력을 확대하는 추세
£구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙ 구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월
앤스로픽에 최대 40억 달러의 투자 계획을 공개
∙ 한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해
4년간 30억 달러 규모의 계약을 체결
∙ 오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei)
남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발
n 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈
AI와 협력을 확대
∙ 마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를
투자하기로 하면서 오픈AI의 지분 49%를 확보했으며, 오픈AI는 마이크로소프트의 애저(Azure)
클라우드 플랫폼을 사용해 AI 모델을 훈련
£구글, 클라우드 경쟁력 강화를 위해 생성 AI 투자 확대
n 구글은 수익률이 높은 클라우드 컴퓨팅 시장에서 아마존과 마이크로소프트를 따라잡고자 생성 AI를
통한 기업 고객의 클라우드 지출 확대를 위해 AI 투자를 지속
{'page': 13.0, 'source'

In [65]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽", search_kwargs={"alpha": 0, "k": 1}
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화
KEY Contents
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인
앤스로픽 및 오픈AI와 협력을 확대하는 추세
£구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙ 구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월
앤스로픽에 최대 40억 달러의 투자 계획을 공개
∙ 한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해
4년간 30억 달러 규모의 계약을 체결
∙ 오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei)
남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발
n 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈
AI와 협력을 확대
∙ 마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를
투자하기로 하면서 오픈AI의 지분 49%를 확보했으며, 오픈AI는 마이크로소프트의 애저(Azure)
클라우드 플랫폼을 사용해 AI 모델을 훈련
£구글, 클라우드 경쟁력 강화를 위해 생성 AI 투자 확대
n 구글은 수익률이 높은 클라우드 컴퓨팅 시장에서 아마존과 마이크로소프트를 따라잡고자 생성 AI를
통한 기업 고객의 클라우드 지출 확대를 위해 AI 투자를 지속
{'page': 13.0, 'source'

동적 `search_kwargs` 사용
- `filter`: metadata 필터링 적용

(예시) `page` 가 5보다 작은 문서만 검색합니다.

In [66]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽의 claude 출시 관련 내용을 알려줘",
    search_kwargs={"filter": {"page": {"$lt": 5}}, "k": 2},
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

2023년 12월호
Ⅰ
. 인공지능 산업 동향 브리프
1. 정책/법제
▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 ·························1
▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의···························2
▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언···························3
▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각·····································4
▹ 미국 연방거래위원회, 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출·················5
▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항···················································6
2. 기업/산업
▹ 미국 프런티어 모델 포럼, 1,000만 달러 규모의 AI 안전 기금 조성································7
▹ 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개 ·······································8
▹ 알리바바 클라우드, 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································9
▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ···························································10
▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································11
▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망···········································12
{

동적 `search_kwargs` 사용
- `filter`: metadata 필터링 적용

(예시) `source` 가 `SPRi AI Brief_8월호_산업동향.pdf` 문서내 검색합니다.

In [67]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽의 claude 3.5 출시 관련 내용을 알려줘",
    search_kwargs={
        "filter": {"source": {"$eq": "SPRi AI Brief_7월호_산업동향.pdf"}},
        "k": 3,
    },
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

### Reranking 적용

아직은 `pre` 기능입니다.

- 동적 reranking 기능을 구현해 놓았지만, pinecone 라이브러리 의존성에 문제가 있을 수 있습니다.
- 따라서, 아래 코드는 향후 의존성 해결 후 원활하게 동작할 수 있습니다.

참고 문서: https://docs.pinecone.io/guides/inference/rerank

In [68]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽의 클로드 소넷",
    search_kwargs={"rerank": True, "rerank_model": "bge-reranker-v2-m3", "top_n": 3},
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

[rerank_documents]


AttributeError: 'GRPCIndex' object has no attribute 'inference'